In [9]:
"""import open3d as o3d
import meshplot as mp
import numpy as np

def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(np.asarray(myMesh1.vertices), np.asarray(myMesh1.triangles), c=cmap1, s=[2, 2, 0])
    mp.subplot(np.asarray(myMesh2.vertices), np.asarray(myMesh2.triangles), c=cmap2, s=[2, 2, 1], data=d)

mesh1 = o3d.io.read_triangle_mesh("/Users/gozdeunver/Downloads/SphereDataset/NoCollider/Force_normal/5.obj")
mesh2 = o3d.io.read_triangle_mesh("/Users/gozdeunver/Downloads/SphereDataset/NoCollider/Force_20_offcenter/5.obj")
print(f'Mesh 1 : {len(np.asarray(mesh1.vertices))} vertices, {len(np.asarray(mesh1.triangles))} faces\n'
      )

double_plot(mesh1,mesh2)"""

### Rotate mesh

In [1]:
import pyvista as pv
import open3d as o3d
import numpy as np
import os
import pandas as pd

In [ ]:
p = pv.Plotter()
p.add_actor(axes.actor)
p.add_text("X-Axis Rotation", font_size=14)
p.camera = camera
p.add_mesh(nondeformed_mesh)
#p.add_point_labels(rot.points[285], [rot.points[285]], point_size=20, font_size=36)
#mesh = pv.Line((0, 10, 0), rot.points[285])
sphere = pv.Sphere(radius=0.01,center=rot.points[21377])

#mesh.plot(color='k', line_width=10)
p.add_mesh(sphere)
p.show()

In [13]:
org_dir="top_diagonal_corner"
nondeformed_org=pv.read(f"/Users/gozdeunver/Downloads/cube/{org_dir}/cube_crusher1.obj").triangulate()
deformed_org=pv.read(f"/Users/gozdeunver/Downloads/cube/{org_dir}/cube_crusher2.obj").triangulate()
deformation_vectors = nondeformed_org.points-deformed_org.points
distances_ = np.linalg.norm(deformation_vectors, axis=1)
deformed_org.points[0,:]=[1.,1.,1.]
deformed_org.points[:2]

pyvista_ndarray([[ 1.      ,  1.      ,  1.      ],
                 [-0.102846,  0.226566,  0.098459]], dtype=float32)

In [94]:
#nondeformed = pv.PolyData(np.asarray(nondeformed_.vertices),np.hstack(np.asarray(nondeformed_.triangles)))
#deformed=pv.PolyData(np.asarray(deformed_.vertices),np.hstack(np.asarray(deformed_.triangles)))
org_dir="top_diagonal_corner"
nondeformed_org=pv.read(f"../cube/{org_dir}/cube_crusher1.obj").triangulate()
farthest_dist=np.max(np.linalg.norm(nondeformed_org.points-nondeformed_org.center,axis=1))
nondeformed=nondeformed_org.scale([1/farthest_dist, 1/farthest_dist, 1/farthest_dist], inplace=False)

deformed_50_org=pv.read(f"../cube/{org_dir}/cube_crusher50.obj").triangulate()
deformed_50=deformed_50_org.scale([1/farthest_dist, 1/farthest_dist, 1/farthest_dist], inplace=False)

deformation_vectors = nondeformed.points-deformed_50.points
distances_ = np.linalg.norm(deformation_vectors, axis=1)
contact_point=np.argmax(distances_)

axes = pv.Axes(show_actor=True,line_width=5)
axes.origin = nondeformed.center
bool_var=False
output_directory=f"./data/cube/{org_dir}_Rotated"
"""camera = pv.Camera()
camera.position = (2.0, 2.0, 2.0)
camera.focal_point = (0., 0., 0.)"""
if not os.path.exists("cube_dataset.csv"):
    f=open("cube_dataset.csv","w+")
    f.write("input,target,contact_vertex,train_sample\n")
else:
    f=open("cube_dataset.csv","a+")
    
for i in range(2,51):

    deformed_org=pv.read(f"../cube/{org_dir}/cube_crusher{i}.obj").triangulate()
    deformed=deformed_org.scale([1/farthest_dist, 1/farthest_dist, 1/farthest_dist], inplace=False)

    for angle in range(0,4):
        rot = deformed.rotate_z(90 * angle, point=axes.origin, inplace=False)
        deformed_path=os.path.join(output_directory,f"cube_crusher{i}_z_angle{angle*90}.ply")
        rot.save(deformed_path)
        rot_non=nondeformed.rotate_z(90 * angle, point=axes.origin, inplace=False)
        non_deformed_path=os.path.join(output_directory,f"cube_crusher1_z_angle{angle*90}.ply")
        rot_non.save(non_deformed_path)
       
        f.write(non_deformed_path+","+deformed_path+","+str(contact_point)+","+str(bool_var)+"\n")

        rot = deformed.rotate_y(90 * angle, point=axes.origin, inplace=False)
        deformed_path=os.path.join(output_directory,f"cube_crusher{i}_y_angle{angle*90}.ply")
        rot.save(deformed_path)
        rot_non=nondeformed.rotate_y(90 * angle, point=axes.origin, inplace=False)
        non_deformed_path=os.path.join(output_directory,f"cube_crusher1_y_angle{angle*90}.ply")
        rot_non.save(non_deformed_path)
        
        f.write(non_deformed_path+","+deformed_path+","+str(contact_point)+","+str(bool_var)+"\n")

        rot = deformed.rotate_x(90 * angle, point=axes.origin, inplace=False)
        deformed_path=os.path.join(output_directory,f"cube_crusher{i}_x_angle{angle*90}.ply")
        rot.save(deformed_path)
        rot_non=nondeformed.rotate_x(90 * angle, point=axes.origin, inplace=False)
        non_deformed_path=os.path.join(output_directory,f"cube_crusher1_x_angle{angle*90}.ply")
        rot_non.save(non_deformed_path)
        
        f.write(non_deformed_path+","+deformed_path+","+str(contact_point)+","+str(bool_var)+"\n")
f.close()


In [95]:
df=pd.read_csv("./cube_dataset.csv")
df.to_csv("./cube_dataset.csv")

In [21]:
"""import pandas as pd
df=pd.read_csv("./cube_dataset.csv")
bool_vec=np.ones(len(df),dtype='bool')
df["train_sample"]=bool_vec
df.loc[df['contact_vertex'] == 6058,["train_sample"]]=False
df.to_csv("./cube_dataset_final.csv")"""

In [29]:
axes = pv.Axes(show_actor=True,line_width=5)
axes.origin = nondeformed_center

In [ ]:
p = pv.Plotter()
p.add_actor(axes.actor)
p.add_text("X-Axis Rotation", font_size=14)
camera = pv.Camera()
camera.position = (10.0, 10.0, 10.0)
camera.focal_point = (0., 0., 0.)
p.camera = camera

p.add_mesh(rot)
#p.add_point_labels(rot.points[285], [rot.points[285]], point_size=20, font_size=36)
#mesh = pv.Line((0, 10, 0), rot.points[285])
sphere = pv.Sphere(radius=0.1,center=rot.points[1066])

#mesh.plot(color='k', line_width=10)
p.add_mesh(sphere)
p.show()
